In [1]:
import scipy.io as sio
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [2]:
load = sio.loadmat('D:\Yew Wah\Downloads\spamData.mat')

# sklearn Gaussian NB answer

In [3]:
x_train = load['Xtrain']
y = preprocessing.scale(x_train)

In [222]:
x = load['Xtrain']
min_max_scaler = preprocessing.StandardScaler()
scaler = min_max_scaler.fit(x)
x_scaled = scaler.transform(x)
df = pd.DataFrame(x_scaled)

In [5]:
clf = GaussianNB()
clf.fit(df.values, load['ytrain'].flatten())

GaussianNB(priors=None)

In [6]:
xtest = scaler.transform(load['Xtest'])
y_pred = clf.predict(xtest)

In [7]:
clf.predict(xtest[:10]) #[0, 1, 0, 1, 0, 0, 1, 1, 0, 0]

array([0, 1, 1, 1, 0, 1, 1, 1, 0, 1], dtype=uint8)

In [8]:
accuracy_score(load['ytest'].flatten(), y_pred)

0.79817708333333337

In [9]:
sum(y_pred)

847

# Building train set

In [223]:
x_scaled

array([[-0.1119685 , -0.17172773, -0.11950585, ..., -0.10338122,
        -0.15317142,  1.68293105],
       [-0.34647496, -0.17172773, -0.5696639 , ..., -0.1071547 ,
        -0.22632329, -0.43277641],
       [-0.34647496, -0.17172773,  1.02635102, ..., -0.10951695,
        -0.22632329, -0.4121431 ],
       ..., 
       [-0.34647496, -0.17172773,  3.13390919, ..., -0.11485504,
        -0.20803532, -0.41849181],
       [-0.34647496, -0.17172773,  4.66853893, ..., -0.1070013 ,
        -0.21260732, -0.42484052],
       [-0.34647496, -0.17172773, -0.5696639 , ..., -0.1120326 ,
        -0.21717931, -0.3280227 ]])

In [281]:
def calc_prob(weights, x):
    expterm = np.nan_to_num(np.exp(weights.dot(x.T)))
    prob = expterm/(1.0 + expterm)
    return prob 

In [282]:
def diff_first(X, prob, labels):
    # flatten labels first
    first_der = X.T.dot(prob - labels)
    return first_der

In [283]:
def hessian(X, prob):
    #Compute S matrix 
    S = prob * (1-prob)
    S_diag = np.diag(S)
    return X.T.dot(S_diag).dot(X)

In [287]:
def newton_update(weights, first_derivative, hessian_matrix, lr = 1):
    print hessian_matrix
    aa = np.linalg.inv(hessian_matrix).dot(first_derivative)
    return weights - aa
    

In [290]:
x = load['Xtrain']
min_max_scaler = preprocessing.StandardScaler()
scaler = min_max_scaler.fit(x)
x_scaled = scaler.transform(x)
wt_b = np.random.randn((x_scaled.shape[1] + 1))
train = np.hstack((x_scaled, np.ones((x_scaled.shape[0], 1))))
labels = load['ytrain']

In [291]:
weights_old = np.zeros(wt_b.shape)
labels = labels.flatten()
while np.linalg.norm(wt_b - weights_old) > 1e-9:
    weights_old = wt_b
    pred = calc_prob(wt_b, train)
    
    first_der = diff_first(train, pred, labels)
    hessian_matrix = hessian(train, pred)
    wt_b = newton_update(wt_b, first_der, hessian_matrix)
    print np.linalg.norm(wt_b - weights_old)

[[ 178.59239824   -0.30768817   17.03837191 ...,   12.23003207
    31.80603028    1.02111668]
 [  -0.30768817   40.43920345    6.9079122  ...,    5.14518597    4.1631081
   -14.62326297]
 [  17.03837191    6.9079122   113.86174964 ...,   14.98960808
    23.00700422  -28.95325542]
 ..., 
 [  12.23003207    5.14518597   14.98960808 ...,   39.48598835
    44.58795005   -7.17038622]
 [  31.80603028    4.1631081    23.00700422 ...,   44.58795005  154.4855215
     4.22499841]
 [   1.02111668  -14.62326297  -28.95325542 ...,   -7.17038622
     4.22499841  207.34228117]]
79.0935763363
[[ 34.04245666  -0.38222757   3.47053029 ...,   1.51889305   5.8682462
    3.04922476]
 [ -0.38222757   0.99780302   0.54766311 ...,   0.594343     0.97113342
   -2.95344547]
 [  3.47053029   0.54766311  17.69308541 ...,   0.6031829    4.02747589
   -0.67585104]
 ..., 
 [  1.51889305   0.594343     0.6031829  ...,   4.94755694   8.17568182
   -0.51262602]
 [  5.8682462    0.97113342   4.02747589 ...,   8.17568182

D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


[[ 0.01038104  0.00514529 -0.00868082 ...,  0.00500026 -0.0017752
  -0.02996189]
 [ 0.00514529  0.00255022 -0.00430258 ...,  0.00247834 -0.00087986
  -0.01485038]
 [-0.00868082 -0.00430258  0.00725906 ..., -0.00418131  0.00148445
   0.02505468]
 ..., 
 [ 0.00500026  0.00247834 -0.00418131 ...,  0.00240849 -0.00085506
  -0.01443181]
 [-0.0017752  -0.00087986  0.00148445 ..., -0.00085506  0.00030357
   0.0051236 ]
 [-0.02996189 -0.01485038  0.02505468 ..., -0.01443181  0.0051236
   0.08647633]]
7.92424659539e+21
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


LinAlgError: Singular matrix

In [ ]:
pred = calc_prob(wt_b, train)

In [159]:
labels = load['ytrain']
cc = pred - labels.flatten()

(3065L, 3065L)

In [164]:
diff_first(train, pred, load['ytrain'].flatten()).shape

(3065L,)


(58L,)

In [265]:
log_p_y0 = np.log(p_y0)
log_p_y1 = np.log(p_y1)

# Computing probabilities of each col aka p(x|y) using Gaussian

In [266]:
import scipy.stats

In [271]:
means = []
def compute_probs(df, alpha, beta, eps = 1e-9):
    means = []
    variances = []
    
    # Assumes that the df already only has the class we want
    N = len(df)
     
    for feat in df.columns[:-1]:
        # Calculate mean and var 
        mean = np.mean(df[feat])
        var = np.var(df[feat])
        means.append(mean)
        variances.append(var)
    return means, variances


In [272]:
alpha = 1
beta = alpha

In [273]:
df_y0 = df_y[df_y[57] == 0]
df_y1 = df_y[df_y[57] == 1]
m_y0, var_y0 = compute_probs(df_y0, alpha, beta)
m_y1, var_y1 = compute_probs(df_y1, alpha, beta)

# Test Data 

In [277]:
xtest[0]

array([-0.34647496, -0.17172773, -0.20135277, -0.05304564,  0.39377563,
       -0.35653076, -0.29526638, -0.29622979, -0.329799  , -0.4184017 ,
       -0.30411241,  0.25134088,  0.90464686, -0.17254418, -0.19649428,
       -0.31118533, -0.32299122, -0.35653777, -0.93684592, -0.18914356,
       -0.69792298, -0.12158892, -0.2881641 , -0.2156358 ,  1.43341923,
       -0.31550767, -0.2254397 , -0.23114582, -0.17413025, -0.21949629,
       -0.17558122, -0.13963826, -0.19096804, -0.14179281, -0.1880593 ,
        0.20542629,  0.10053089, -0.06592702, -0.18246718, -0.18416381,
       -0.12151107, -0.17162805,  0.63278676, -0.14079339, -0.11824356,
       -0.18291581, -0.06485205, -0.11807012, -0.15912838, -0.30309409,
       -0.14531673, -0.3038868 ,  0.15016725, -0.10378077, -0.09040414,
       -0.15774341, -0.10105634])

In [278]:
df_test = pd.DataFrame(xtest)

In [279]:
np.mean(df_test[0])

0.011348047809125489

# Classify Data 

In [301]:
from math import e, sqrt, pi
def gaussian(x, mean, var):
    return 1/(sqrt(2*pi)*var)*e**(-0.5*(float(x-mean)/var)**2)

In [305]:
def compute_prob_for_class(df_test, mean, var, label):
    classes = []
    for idx, row in df_test.iterrows():
        prob = 0
        for col_index in range(len(row)):
            prob += np.log(np.nan_to_num(gaussian(row[col_index], mean[col_index], var[col_index])) + 1e-9)
        classes.append(prob + label)      
        
    return classes
y_pred_0 = compute_prob_for_class(df_test, m_y0, var_y0, log_p_y0)
y_pred_1 = compute_prob_for_class(df_test, m_y1, var_y1, log_p_y1)

In [306]:
final_res = zip(y_pred_0, y_pred_1)

In [307]:
np.nan_to_num(-np.inf)

-1.7976931348623157e+308

In [309]:
final_res

[(-35.298572698116004, -107.01823681179475),
 (-62.804963339026685, -59.769777053692714),
 (-73.540287284976557, -99.037564584027294),
 (-157.54190387884111, -87.963501688108863),
 (-37.586014592096163, -161.73474094757776),
 (-46.047610129643438, -100.2846080126882),
 (-138.17491941053854, -105.31593594188003),
 (-190.95114977899388, -79.767312897616478),
 (-59.993788807575562, -102.50702188260887),
 (-71.263159496923137, -81.315196010518946),
 (-122.60965861221814, -100.84041897220773),
 (-46.529628201006531, -163.67241791318332),
 (-117.65439119648654, -109.57441482884914),
 (-101.74202932621309, -312.41570193079014),
 (-79.83735161333351, -83.288533821019016),
 (-185.99132566257424, -66.588412887009824),
 (-119.72543255769337, -61.001240245877938),
 (-86.598821194765378, -77.369523831644059),
 (-51.066302168979895, -144.152679165894),
 (-58.452996149975391, -60.112962931929061),
 (-74.15468377359376, -81.570426152285478),
 (-61.86202607646868, -159.51621493290307),
 (-65.3562549000

In [310]:
final_res = [0 if x[0] >= x[1] else 1 for x in final_res]
final_res

[0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,


In [311]:
final_res[:10]

[0, 1, 0, 1, 0, 0, 1, 1, 0, 0]

# Check res 

In [312]:
sum(final_res)

660

In [313]:
grd_truth = load['ytest'].flatten().tolist()

In [314]:
sum(grd_truth)

581

In [315]:
np.sum(load['ytest'])

581

In [316]:
pred = zip(grd_truth, final_res)

In [317]:
acc = [1 if int(x[0])==x[1] else 0 for x in pred]

In [318]:
sum(acc)/float(len(grd_truth))

0.8157552083333334

In [250]:
final_res[20:40]

[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0]

In [251]:
len(grd_truth) - sum(acc)

283

In [ ]:
sum(final_res)

In [289]:
1-00.8157552083333334

0.18424479166666663